In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys

sys.path.append("../")
import copy

import logging
from src.utils import logging_utils
from src import functional
from src.models import ModelandTokenizer
from src.dataset import load_dataset, load_relation, fill_template


logger = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.INFO,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)

In [3]:
from src.models import ModelandTokenizer

model_name = "meta-llama/Llama-2-7b-hf"
mt = ModelandTokenizer(model_path=model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

2023-12-12 19:20:15 src.models INFO     loaded model <meta-llama/Llama-2-7b-hf> | size: 12980.516 MB


In [4]:
relation = load_relation(
    relation_file = "head_of_government.json",
    num_icl = 0,                          # initialize with zero-shot
    default_path="../data",
    batch_size=500
)

all_samples = copy.deepcopy(relation.samples)

# filter zero-shot model knowledge
relation = functional.filter_samples_by_model_knowledge(
    mt = mt,
    relation = relation,
)

relation.properties["num_icl"] = 5
relation.select_icl_examples(num_icl=5)

relation.samples = all_samples

# filter model knowledge with `num_icl` shots
relation = functional.filter_samples_by_model_knowledge(
    mt = mt,
    relation = relation,
)

2023-12-12 19:20:16 src.dataset INFO     initialized relation -> "head of government" with 500 samples
2023-12-12 19:20:30 src.functional INFO     filtered relation "head of government" to 183 samples (with 0-shots)
2023-12-12 19:20:47 src.functional INFO     filtered relation "head of government" to 360 samples (with 5-shots)


In [8]:
from relations.src.operators import JacobianIclMeanEstimator
import relations.src.functional as relations_functional

# relations_functional.make_prompt = functional.make_prompt